<a href="https://colab.research.google.com/github/shakeeluetp1041/Machine-Learning/blob/main/Recurrent_Neural_Network_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Vanilla RNN for MNIST Digit Classification
# You can use the same code for GRU, Just replace nn.RNN------>nn.GRU in line 27
# You can use the same code for LSTM, Just replace nn.RNN------>nn.LSTM in line 27. Also replace line 32 with 33 by uncommenting

import torch
from torch.utils.data import DataLoader
import torchvision
from torchvision import transforms,datasets
import matplotlib.pyplot as plt
from torch import nn
# To remove the directory
# import shutil
# shutil.rmtree('./data')
# Device agnositic code
device='cuda' if torch.cuda.is_available() else 'cpu'
train_data=torchvision.datasets.MNIST(root='data',train=True,download=True,transform=transforms.ToTensor())
test_data=torchvision.datasets.MNIST(root='data',train=False,download=True,transform=transforms.ToTensor())
BATCH_SIZE=64
train_dataloader=DataLoader(dataset=train_data,batch_size=BATCH_SIZE,shuffle=True)
test_dataloader=DataLoader(dataset=test_data,batch_size=BATCH_SIZE,shuffle=False)
class RNN(nn.Module):
  def __init__(self,inputsize,hidden_size,num_layers,seq_length,num_classes):# No need to define seq_lenth, it can be extracted form the shape of input
    super(RNN,self).__init__()
    self.hidden_size=hidden_size
    self.num_layers=num_layers
    self.seq_length=seq_length
    self.rnn=nn.RNN(input_size=inputsize,hidden_size=hidden_size,num_layers=num_layers,batch_first=True)
    self.fc=nn.Linear(seq_length*hidden_size,num_classes)
  def forward(self,x):
    h0=torch.zeros(self.num_layers,x.shape[0],self.hidden_size).to(device)
    c0=torch.zeros(self.num_layers,x.shape[0],self.hidden_size).to(device)
    rnn_output,_=self.rnn(x,h0)      # comment  this for nn.LSTM
    # rnn_output,_=self.rnn(x,(h0,c0)) # uncomment this for nn.LSTM

    output=rnn_output.reshape(-1,self.seq_length*self.hidden_size)
    output=self.fc(output)
    return output
def accuracy_fn(y_pred,y_true):
  count=(y_pred==y_true).sum().item()
  acc=count/len(y_true)
  return (acc*100)
model=RNN(inputsize=28,hidden_size=256,num_layers=2,seq_length=28,num_classes=len(train_data.classes)).to(device)
loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.001)


EPOCHS=4
for epoch in range(EPOCHS):
  model.train()
  train_loss=0
  train_acc=0
  test_loss=0
  test_acc=0
  for (X_train,y_train) in train_dataloader:
    X_train,y_train=X_train.to(device),y_train.to(device)
    train_output=model(X_train.squeeze())
    loss=loss_fn(train_output,y_train)
    train_loss+=loss
    y_train_pred=torch.argmax(train_output,dim=1)
    train_acc+=accuracy_fn(y_train_pred,y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  train_loss/=len(train_dataloader)
  train_acc/=len(train_dataloader)
  model.eval()
  with torch.inference_mode():
    for (X_test,y_test) in test_dataloader:
      X_test,y_test=X_test.to(device),y_test.to(device)
      test_output=model(X_test.squeeze())
      loss=loss_fn(test_output,y_test)
      test_loss+=loss
      y_test_pred=torch.argmax(test_output,dim=1)
      test_acc+=accuracy_fn(y_test_pred,y_test)
    test_loss/=len(test_dataloader)
    test_acc/=len(test_dataloader)
  print(f'EPOCH: {epoch}, TRAIN LOSS: {train_loss:.4f}, TRAIN ACCURACY: {train_acc:.2f},TEST Loss: {test_loss:.4f}, Test ACCURACY: {test_acc:.2f}')





Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:01<00:00, 8009839.22it/s] 


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 521698.12it/s]


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 4309756.05it/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 4225000.84it/s]


Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw

EPOCH: 0, TRAIN LOSS: 0.2810, TRAIN ACCURACY: 91.47,TEST Loss: 0.1543, Test ACCURACY: 95.39
EPOCH: 1, TRAIN LOSS: 0.1285, TRAIN ACCURACY: 96.11,TEST Loss: 0.1188, Test ACCURACY: 96.29
EPOCH: 2, TRAIN LOSS: 0.1136, TRAIN ACCURACY: 96.58,TEST Loss: 0.1014, Test ACCURACY: 96.96
EPOCH: 3, TRAIN LOSS: 0.1017, TRAIN ACCURACY: 97.04,TEST Loss: 0.1095, Test ACCURACY: 96.87
